In [297]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
import numpy as np
from sklearn.cross_validation import train_test_split

In [298]:
data_train = pd.read_csv('data/train_MNIST.csv')

In [299]:
X_tr = data_train.drop('label',axis=1)
y_tr = data_train.label

In [300]:
# Need to scale features in order for the softmax to learn under certain set param values, here the scaling range is 0~1
scaler = MinMaxScaler(copy=True)
scaler.fit(X_tr)
X_tr_scaled = scaler.transform(X_tr)

In [301]:
x = tf.placeholder(tf.float32,[None,784])

In [302]:
#initail weight, real value to be learned
W = tf.Variable(tf.zeros([784,10]))
#initial bias, real value to be learned
b = tf.Variable(tf.zeros([10]))

In [303]:
y = tf.nn.softmax(tf.matmul(x,W)+b)

In [304]:
# transform y_tr from numerical labels to one_hot code format
y_tr[:5]
#y_tr_code = pd.get_dummies(y_tr)
y_tr_code = np.array(pd.get_dummies(y_tr))

In [305]:
X_train = X_tr_scaled[:32000]
X_test = X_tr_scaled[32000:]
y_train = y_tr_code[:32000]
y_test = y_tr_code[32000:]

In [306]:
# define cross_entropy
y_ = tf.placeholder(tf.float32,[None,10])
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_*tf.log(y),reduction_indices=[1]))

In [83]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [664]:
init = tf.initialize_all_variables()

In [665]:
sess = tf.Session()
sess.run(init)

In [640]:
#mnist = input_data.read_data_sets("MNIST_data", one_hot=True)

In [642]:
#_, loss_val = sess.run([train_step,cross_entropy], feed_dict={x: X_tr_scaled[0:100], y_: y_tr_code[0:100]})
#_, loss_val = sess.run([train_step,cross_entropy], feed_dict={x: X_tr_scaled[100:200], y_: y_tr_code[100:200]})
#_, loss_val = sess.run([train_step,cross_entropy], feed_dict={x: X_tr[200:300], y_: y_tr_code[200:300]})
#_, loss_val = sess.run([train_step,cross_entropy], feed_dict={x: X_tr[300:400], y_: y_tr_code[300:400]})
#_, loss_val = sess.run([train_step,cross_entropy], feed_dict={x: X_tr[400:500], y_: y_tr_code[400:500]})
#print loss_val

2.01386


In [666]:
for i in range(1000):
    batch_xs = X_tr_scaled[i*100:(i+1)*100]
    batch_ys = y_tr_code[i*100:(i+1)*100]
    _, loss_val = sess.run([train_step,cross_entropy], feed_dict={x: batch_xs, y_: batch_ys})
    #print loss_val

In [667]:
#for i in range(2):
    #batch_xs, batch_ys = mnist.train.next_batch(100)
    #_, loss_val = sess.run([train_step,cross_entropy], feed_dict={x: batch_xs, y_: batch_ys})
    ##print loss_val

In [668]:
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))

In [669]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [670]:
print(sess.run(accuracy, feed_dict={x: X_tr_scaled, y_: y_tr_code}))

#print(sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.91031


**91% is pretty low, especially for neural network(a simple knn model can get 97%!)**

##To get a more respectable result, trying convolutional NN##

In [307]:
def weight_variable(shape):
    initial = tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [308]:
def conv2d(x, W):
  return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
  return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [309]:
# First convolutional layer
W_conv1 = weight_variable([5, 5, 1, 32])
b_conv1 = bias_variable([32])

In [310]:
x_image = tf.reshape(x, [-1,28,28,1])

In [311]:
h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

In [312]:
# Second convolutional layer
W_conv2 = weight_variable([5, 5, 32, 64])
b_conv2 = bias_variable([64])

In [313]:
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

In [314]:
# Fully connected layer
W_fc1 = weight_variable([7 * 7 * 64, 1024])
b_fc1 = bias_variable([1024])

In [315]:
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

In [316]:
# Add dropout to prevent overfitting
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

In [359]:
# Readout layer (softmax)
W_fc2 = weight_variable([1024, 10])
b_fc2 = bias_variable([10])

y_conv=tf.nn.softmax(tf.matmul(h_fc1_drop, W_fc2) + b_fc2)

In [360]:
predict = tf.argmax(y_conv,1)

In [361]:
# Start training
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y_conv), reduction_indices=[1]))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [362]:
init = tf.initialize_all_variables()
sess = tf.InteractiveSession()
sess.run(init)

In [363]:
epochs_completed = 0
index_in_epoch = 0
num_examples = X_train.shape[0]
train_images = X_train
train_labels = y_train

# Introduce next_batch function to serve data by batches on a rolling basis
def next_batch(batch_size):
    
    global train_images
    global train_labels
    global index_in_epoch
    global epochs_completed
    
    start = index_in_epoch
    index_in_epoch += batch_size
    
    # when all trainig data have been already used, it is reorder randomly    
    if index_in_epoch > num_examples:
        # finished epoch
        epochs_completed += 1
        # shuffle the data
        perm = np.arange(num_examples)
        np.random.shuffle(perm)
        train_images = X_train[perm]
        train_labels = y_train[perm]
        # start next epoch
        start = 0
        index_in_epoch = batch_size
        assert batch_size <= num_examples
    end = index_in_epoch
    return train_images[start:end], train_labels[start:end]

In [364]:
for i in range(20000):
    batch_size = 50
    batch_xs, batch_ys = next_batch(batch_size) 
    #batch_xs = X_tr_scaled[i*batch_size:(i+1)*batch_size]
    #batch_ys = y_tr_code[i*batch_size:(i+1)*batch_size]
    if i%200 == 0:
        train_accuracy = accuracy.eval(feed_dict={x: batch_xs, y_: batch_ys, keep_prob:1.0})
        print("step %d, training accuracy %g"%(i, train_accuracy))
        #print predict.eval(feed_dict={x: batch_xs[1 : 10], keep_prob: 1.0})
    train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob:0.5})

step 0, training accuracy 0.06
step 200, training accuracy 0.94
step 400, training accuracy 0.94
step 600, training accuracy 0.94
step 800, training accuracy 0.9
step 1000, training accuracy 0.94
step 1200, training accuracy 1
step 1400, training accuracy 0.94
step 1600, training accuracy 0.94
step 1800, training accuracy 0.96
step 2000, training accuracy 1
step 2200, training accuracy 1
step 2400, training accuracy 1
step 2600, training accuracy 0.96
step 2800, training accuracy 0.98
step 3000, training accuracy 1
step 3200, training accuracy 1
step 3400, training accuracy 0.94
step 3600, training accuracy 1
step 3800, training accuracy 0.98
step 4000, training accuracy 1
step 4200, training accuracy 1
step 4400, training accuracy 1
step 4600, training accuracy 1
step 4800, training accuracy 1
step 5000, training accuracy 1
step 5200, training accuracy 1
step 5400, training accuracy 0.98
step 5600, training accuracy 1
step 5800, training accuracy 1
step 6000, training accuracy 0.96
st

In [365]:
print("test accuracy %g"%accuracy.eval(feed_dict={
    x: X_test, y_: y_test, keep_prob: 1.0}))

test accuracy 0.9905


**Load testing data**

In [338]:
X_te = pd.read_csv('data/test_MNIST.csv')
X_te['ImageId'] = X_te.index.values+1

In [339]:
submit = pd.DataFrame(X_te.ImageId, columns=['ImageId'])

In [340]:
X_te.drop('ImageId',axis=1,inplace=True)

In [341]:
scaler = MinMaxScaler(copy=True)
scaler.fit(X_te)
X_te_scaled = scaler.transform(X_te)

In [342]:
predicted_lables = np.zeros(X_te_scaled.shape[0])

In [343]:
for i in range(0,X_te_scaled.shape[0]//50):
    predicted_lables[i*50 : (i+1)*50] = predict.eval(feed_dict={x: X_te_scaled[i*50 : (i+1)*50], keep_prob: 1.0})

In [344]:
predicted_lables[:10]

array([ 2.,  0.,  9.,  9.,  3.,  7.,  0.,  3.,  0.,  3.])

In [345]:
predicted_lables[:10]

array([ 2.,  0.,  9.,  9.,  3.,  7.,  0.,  3.,  0.,  3.])

In [346]:
submit['Label'] = predicted_lables

In [347]:
submit.Label = submit.Label.astype(int)

In [348]:
submit[:5]

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [349]:
submit.to_csv('submisson_tensor_CNN.csv',index=False)